#### Name: Kalp Ranpura
#### Mail: kalpmrr@duck.com
#### Topic: AlgoBulls_Pythob_Developer_Coding_Assignment
#### Date: 15-06-2023

# Part-1

In [ ]:
import pandas as pd
from alpha_vantage.timeseries import TimeSeries
import plotly.graph_objects as go

In [ ]:
class ScriptData:
    used_symbols = set()  # set to keep track of used symbols

    def __init__(self):
        self.api_key = 'M4N0W0ER2FCVUBW2'
        self.ts = TimeSeries(key=self.api_key, output_format='pandas')
        self.data = None

    def fetch_intraday_data(self, symbol):
        self.data, _ = self.ts.get_intraday(symbol=symbol)  

    def convert_intraday_data(self, symbol):
        df = self.data

        df.index = pd.to_datetime(df.index)
        df.reset_index(inplace=True)
        df.columns = ['timestamp', 'open', 'high', 'low', 'close', 'volume']
        df['volume'] = df['volume'].astype(int)
        df[['open', 'high', 'low', 'close']] = df[['open', 'high', 'low', 'close']].astype(float)
        self.df = df

    def __getitem__(self, symbol):
        self.used_symbols.add(symbol)
        return self.df

    def __setitem__(self, symbol):
        raise NotImplementedError("Setting data is not supported.")

    def __contains__(self, symbol):
        return symbol in self.used_symbols


In [ ]:
script_data = ScriptData()
script_data.fetch_intraday_data('amzn')
script_data.convert_intraday_data('amzn')
script_data['amzn']


In [ ]:
'amzn' in script_data


# Part-2

In [ ]:
def indicator1(df, timeperiod):
    indicator_df = pd.DataFrame()
    indicator_df['timestamp'] = df['timestamp']
    indicator_df['indicator'] = df['close'].rolling(timeperiod).mean()
    return indicator_df

In [ ]:
indicator1(script_data['amzn'],timeperiod=3)

# Part-3

In [ ]:
class Strategy:
    def __init__(self, symbol):
        self.symbol = symbol
    
    def get_script_data(self):
        # FETCHING DATA WITH API AND CONVERTING IT TO DATAFRAME
        script_data = ScriptData()
        script_data.fetch_intraday_data(self.symbol)
        script_data.convert_intraday_data(self.symbol)
        self.close_df = script_data[self.symbol]
        
        # CREATING INDICATOR DF FOR MOVING AVERAGE
        indicator_data = indicator1(script_data[self.symbol], timeperiod=5)
        self.indic = indicator_data
        
    # METHOD TO GET SIGNAL BASED ON THE CUTS   
    def get_signals(self):  
        signal_df = pd.merge(self.close_df, self.indic, on='timestamp')
        signal_df['signal'] = 'NO_SIGNAL'
        
        # LOGIC FOR CUT_DOWNWARDS
        buy_coincide_indices = signal_df[(signal_df['close'] == signal_df['indicator']) & (signal_df['indicator'].shift(-1) < signal_df['close'].shift(-1))].index
        signal_df.loc[buy_coincide_indices, 'signal'] = 'BUY' 
        
        # LOGIC FOR CUT_UPWARDS
        sell_coincide_indices = signal_df[(signal_df['close'] == signal_df['indicator']) & (signal_df['indicator'].shift(-1) > signal_df['close'].shift(-1))].index
        signal_df.loc[sell_coincide_indices, 'signal'] = 'SELL' 
        
        # FILTERING TO SHOW THE REQUIRED COLUMN
        signals = signal_df.query("signal in ['BUY', 'SELL']")[['timestamp', 'signal']] 
        
        return signals


In [ ]:
df=Strategy('amzn')
df.get_script_data()
df.get_signals()

# Part-4

In [ ]:
class Plots:
    
    def __init__(self, symbol):
        self.symbol = symbol
    
    def get_dfs(self):
        script_data = ScriptData()
        script_data.fetch_intraday_data(self.symbol)
        script_data.convert_intraday_data(self.symbol)
        self.close_df = script_data[self.symbol]
        
        self.indicator_df = indicator1(self.close_df, timeperiod=5)
        strategy = Strategy(self.symbol)
        strategy.get_script_data()
        self.signal = strategy.get_signals()
        
    def get_plots(self):
        fig1 = go.Figure()

        # Add candlestick trace

        # Add moving average trace
        fig1.add_trace(go.Scatter(x=self.indicator_df['timestamp'],
                                 y=self.indicator_df['indicator'],
                                 mode='lines',
                                 name='Moving Average'))

        fig1.add_trace(go.Scatter(x=self.close_df['timestamp'],
                                 y=self.close_df['close'],
                                 mode='lines',
                                 name='Closing'))

        # Customize the layout
        fig1.update_layout(title='Closing Price with Moving Average',
                          xaxis_title='Date',
                          yaxis_title='Price')

        # Annotating the layout with signals
        annotations = []

        for i, row in self.signal.iterrows():
            timestamp = row['timestamp']
            close_value = self.close_df.loc[self.close_df['timestamp'] == timestamp, 'close'].values[0]

            annotation = dict(
                x=timestamp,
                y=close_value,
                xref='x',
                yref='y',
                text=row['signal'],
                showarrow=True,
                arrowhead=2,
                arrowsize=1.5,
                arrowwidth=1,
                arrowcolor='black',
            )

            annotations.append(annotation)

        fig1.update_layout(annotations=annotations)
        # Display the chart
        fig1.show()

        # Another chart
        fig2 = go.Figure()

        # Add candlestick trace
        fig2.add_trace(go.Candlestick(x=self.close_df['timestamp'],
                                     open=self.close_df['open'],
                                     high=self.close_df['high'],
                                     low=self.close_df['low'],
                                     close=self.close_df['close'],
                                     name='Candlestick'))
        fig2.add_trace(go.Scatter(x=self.indicator_df['timestamp'],
                                 y=self.indicator_df['indicator'],
                                 mode='lines',
                                 name='Moving Average'))
        # Customize the layout
        fig2.update_layout(title='Candlestick Chart with Moving Average',
                          xaxis_title='Date',
                          yaxis_title='Price')
        # Display the chart
        fig2.show()


In [ ]:
plot=Plots('AMZN')
plot.get_dfs()
plot.get_plots()